# We will test the Time-shift 1000
- We will use the vanilla model-Baseline (No augmentation)
- We will use the 1000ms augmentation model here

In [5]:
import numpy as np
import scipy.io

from keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint


import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten, Activation
from keras.layers.core import Permute, Reshape
from keras import backend as K

from matplotlib import pyplot as plt
from PIL import Image as img_PIL


# random seed.
rand_seed = 2

from numpy.random import seed
seed(rand_seed)
from tqdm.keras import TqdmCallback
import tensorflow
tensorflow.random.set_seed(rand_seed)

## 1. Loading the data

In [6]:
def one_hot_encoding(y_data):
    Mapping=dict()
    sub_dirs=['downstair','upstair','run','jump','walk','handwashing','exercise']

    categories=7

    count=0
    for i in sub_dirs:
        Mapping[i]=count
        count=count+1

    y_features2=[]
    for i in range(len(y_data)):
        Type=y_data[i]
        lab=Mapping[Type]
        y_features2.append(lab)

    y_features=np.array(y_features2)
    y_features=y_features.reshape(y_features.shape[0],1)
    from keras.utils import to_categorical
    y_features = to_categorical(y_features)

    return y_features

In [7]:
path='auto_data/'


def get_train_data(path=path):
    Train_data=np.load(path+'train_5000.npz')
    Features_imu=np.asarray(Train_data['arr_0'],dtype=np.float64)
    Labels=Train_data['arr_1']
    Features_sound=Train_data['arr_2']

    Labels = one_hot_encoding(Labels)
    Features_imu = Features_imu.reshape(Features_imu.shape[0],1, Features_imu.shape[1], Features_imu.shape[2]) 

    return Features_imu,Labels,Features_sound

def get_valid_data(path=path):
    Train_data=np.load(path+'valid_1000.npz')
    Features_imu=np.asarray(Train_data['arr_0'],dtype=np.float64)
    Labels=Train_data['arr_1']
    Features_sound=Train_data['arr_2']

    Labels = one_hot_encoding(Labels)
    Features_imu = Features_imu.reshape(Features_imu.shape[0],1, Features_imu.shape[1], Features_imu.shape[2]) 

    return Features_imu,Labels,Features_sound

def get_test_data(path=path):
    Train_data=np.load(path+'test_1377.npz')
    Features_imu=np.asarray(Train_data['arr_0'],dtype=np.float64)
    Labels=Train_data['arr_1']
    Features_sound=Train_data['arr_2']

    Labels = one_hot_encoding(Labels)
    Features_imu = Features_imu.reshape(Features_imu.shape[0],1, Features_imu.shape[1], Features_imu.shape[2]) 

    return Features_imu,Labels,Features_sound


In [8]:
Features_imu,Labels,Features_sound = get_train_data()
print(Features_imu.shape, Labels.shape, Features_sound.shape)
print('Train Classes distribution: ',np.sum(Labels, axis =0))

Features_imu2,Labels2,Features_sound2 = get_valid_data()
print(Features_imu2.shape, Labels2.shape, Features_sound2.shape)
print('Valid Classes distribution: ',np.sum(Labels2, axis =0))

Features_imu3,Labels3,Features_sound3 = get_test_data()
print(Features_imu3.shape, Labels3.shape, Features_sound3.shape)
print('Test Classes distribution: ',np.sum(Labels3, axis =0))

(5000, 1, 40, 12) (5000, 7) (5000, 193)
Train Classes distribution:  [760. 994. 546. 780. 532. 732. 656.]
(1000, 1, 40, 12) (1000, 7) (1000, 193)
Valid Classes distribution:  [150. 188. 136. 141.  98. 157. 130.]
(1377, 1, 40, 12) (1377, 7) (1377, 193)
Test Classes distribution:  [219. 268. 124. 146. 143. 234. 243.]


In [ ]:
model_path = 'IMU_Audio_Fusion_1000ms'

from keras.models import load_model

model = load_model(model_path)

In [ ]:
## Test Accuracy
model.evaluate([Features_sound3,Features_imu3],Labels3)

## Observation: All of these test labels are continuous.
### Time window used to create a sample: 2 Second.

# Time shifting, our goal is to randomly insert the samples of different continuous windows in between which is to simulate the real-time deployment setting

# First preprocess the data to get the continuous acc and gyro windows
- data ordering: 12 sensors each with 40 samples.
- 12 sensors are: acc_right, gyro_right, acc_left, gyro_left

In [ ]:
def get_test_data2(path=path):
    Train_data=np.load(path+'test_1377.npz')
    Features_imu=np.asarray(Train_data['arr_0'],dtype=np.float64)
    Labels=Train_data['arr_1']
    Features_sound=Train_data['arr_2']
    
    Features_imu = Features_imu.reshape(Features_imu.shape[0],1, Features_imu.shape[1], Features_imu.shape[2]) 

    return Features_imu,Labels,Features_sound

In [ ]:
Features_imu_t1,Labels_t1,Features_sound_t1 = get_test_data2()
print(Features_imu_t1.shape, Labels_t1.shape, Features_sound_t1.shape)

In [ ]:
# We see all of the labels are continuous
Labels_t1 = np.array(Labels_t1)

In [ ]:
Labels_t1[:100]

# Getting windows of data together with num_windows_together in each

In [ ]:
# For 10 second together
num_windows_together = 5

curr_indx = 0
final_indx = Features_imu_t1.shape[0]

data_together = []

while curr_indx <= final_indx:
    IMU_together   = Features_imu_t1[curr_indx:curr_indx+num_windows_together]
    sound_together = Features_sound_t1[curr_indx:curr_indx+num_windows_together]
    labels_together = Labels_t1[curr_indx:curr_indx+num_windows_together]
    
    data_together.append([IMU_together,sound_together,labels_together])
    
    curr_indx = curr_indx+num_windows_together
    

# Now we randomly distribute the windows

In [ ]:
rand_seed = 4
from numpy.random import seed
seed(rand_seed)

ordering = np.arange(len(data_together))
np.random.shuffle(ordering)

In [ ]:
ordering[:100]

## Now we again recreate the test data

In [ ]:
indx = ordering[0]

Features_imu_t1  = data_together[indx][0]
Features_sound_t1 = data_together[indx][1]
Labels_t1 = data_together[indx][2]


#print(Features_imu_t1.shape,Labels_t1.shape,Features_sound_t1.shape)

for i in range(1, len(data_together)):
    indx = ordering[i]
    IMU_together  = data_together[indx][0]
    sound_together= data_together[indx][1]
    labels_together=data_together[indx][2]
    #print(indx, IMU_together.shape,sound_together.shape,labels_together.shape)
    Features_imu_t1 = np.concatenate((Features_imu_t1,IMU_together),axis=0)
    Labels_t1 = np.concatenate((Labels_t1,labels_together),axis=0)
    Features_sound_t1 = np.concatenate((Features_sound_t1,sound_together),axis=0)
    #print(indx, IMU_together.shape,sound_together.shape,labels_together.shape)

In [ ]:
print(Features_imu_t1.shape, Labels_t1.shape, Features_sound_t1.shape)

In [ ]:
Labels_t1[:100]

In [ ]:
Labels_t1 = one_hot_encoding(Labels_t1)

# Testing accuracy on Reshuffled data.
## Note: not time errors are introduced so Accuracy should be same as the fusion model on original continuous test data

In [ ]:
model.evaluate([Features_sound_t1,Features_imu_t1],Labels_t1)

# Now aggregating the data and doing the timing errors

In [ ]:
acc_right=[]
gyro_right=[]
acc_left=[]
gyro_left=[]

for i in range(Features_imu_t1.shape[0]):
    acc_right.append([Features_imu_t1[i,0,:,0],Features_imu_t1[i,0,:,1],Features_imu_t1[i,0,:,2]])
    gyro_right.append([Features_imu_t1[i,0,:,3],Features_imu_t1[i,0,:,4],Features_imu_t1[i,0,:,5]])
    acc_left.append([Features_imu_t1[i,0,:,6],Features_imu_t1[i,0,:,7],Features_imu_t1[i,0,:,8]])
    gyro_left.append([Features_imu_t1[i,0,:,9],Features_imu_t1[i,0,:,10],Features_imu_t1[i,0,:,11]])
        

In [ ]:
acc_right=np.array(acc_right)
gyro_right=np.array(gyro_right)
acc_left=np.array(acc_left)
gyro_left=np.array(gyro_left)

In [ ]:
print(acc_right.shape)
print(gyro_right.shape)
print(acc_left.shape)
print(gyro_left.shape)

In [ ]:
acc_right_cont= acc_right[0,]
gyro_right_cont= gyro_right[0,]

acc_left_cont= acc_left[0,]
gyro_left_cont= gyro_left[0,]


for i in range(1, Features_imu_t1.shape[0]):
    #print(i)
    acc_right_cont=np.hstack((acc_right_cont,acc_right[i,]))
    gyro_right_cont=np.hstack((gyro_right_cont,gyro_right[i,]))
    acc_left_cont=np.hstack((acc_left_cont,acc_left[i,]))
    gyro_left_cont=np.hstack((gyro_left_cont,gyro_left[i,]))

In [ ]:
print(acc_right_cont.shape)
print(gyro_right_cont.shape)
print(acc_left_cont.shape)
print(gyro_left_cont.shape)


# Now shifting the samples

# Defining function to do the evaluation

In [ ]:
def get_data_sample_shift(shift_samples = 1):
    sample_size = 40 #need to be 40, as decided during training
    total_samples = acc_right_cont.shape[1]
    
    #print(total_samples)
    
    current_cursor = shift_samples

    i = 0

    acc_right_pro= np.array(acc_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    #print(acc_right_pro.shape)

    gyro_right_pro= np.array(gyro_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    acc_left_pro= np.array(acc_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    gyro_left_pro= np.array(gyro_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])

    #print(np.array(acc_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size]).shape)

    while current_cursor<=(total_samples-2*sample_size):
        current_cursor = current_cursor + sample_size
        #print(current_cursor," : ", i)
        a=acc_right_pro
        b=np.array(acc_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        acc_right_pro = np.concatenate((a,b),axis=0)


        a=gyro_right_pro
        b=np.array(gyro_right_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        gyro_right_pro = np.concatenate((a,b),axis=0)


        a=acc_left_pro
        b=np.array(acc_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        #print(a.shape,b.shape)
        acc_left_pro = np.concatenate((a,b),axis=0)

        a=gyro_left_pro
        b=np.array(gyro_left_cont[np.newaxis,:,current_cursor:current_cursor+sample_size])
        gyro_left_pro = np.concatenate((a,b),axis=0)
        i = i+1
        
    IMU_processed = np.concatenate((acc_right_pro,gyro_right_pro,acc_left_pro,gyro_left_pro),axis=1)
    IMU_processed = IMU_processed[:,np.newaxis,:,:]
    IMU_processed = np.swapaxes(IMU_processed,2,3)
        
    size = IMU_processed.shape[0]
        
    return Features_sound_t1[:size], IMU_processed ,Labels_t1[:size]
        


In [ ]:
Accuracy_vanilla = []
Accuracy_1000ms = []

In [ ]:
model_path1 = 'Baseline_IMU_Audio_Fusion'
model_path2 = 'IMU_Audio_Fusion_1000ms'

from keras.models import load_model
model1 = load_model(model_path1)
model2 = load_model(model_path2)

In [ ]:
for shift in range(41):
    Features_sound, IMU ,Labels =  get_data_sample_shift(shift_samples = shift)
    evaluation1 = model1.evaluate([Features_sound,IMU],Labels)
    Accuracy_vanilla.append(evaluation1[1])
    
    evaluation2 = model2.evaluate([Features_sound,IMU],Labels)
    Accuracy_1000ms.append(evaluation2[1])
    
    print("Shift is:",shift,evaluation1[1],evaluation2[1])

In [ ]:
Accuracy_vanilla

In [ ]:
Accuracy_1000ms

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(10,5))
plt.rcParams.update({'font.size': 18})

y = np.arange(len(Accuracy_vanilla[:41]))

y = y * 50

plt.plot(y,(np.array(Accuracy_1000ms[:41])*100))
plt.plot(y,(np.array(Accuracy_vanilla[:41])*100))

#plt.title('Fused Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Time Synchronization Error (ms)')
plt.legend(['Augmentation 1000ms','No Augmentation'], loc='lower left')
plt.show()